In [24]:
#requirements
# ! pip install xgboost scikit-learn

In [25]:
import sklearn
import xgboost as xgb 
import pandas as pd

In [26]:
# preprocessing steps from previous notebook
df = pd.read_excel("./Curated_data_1234_datapoints.xlsx")
df.columns = df.columns.str.strip()
df = df.drop(columns=["Reference DOI", "Unnamed: 23"], axis=1)
df['Support Loading [wt.%]'] = 100 - df['Metal Loading [wt.%]'] - df['Promoter 1 loading [wt.%]'] - df['Promoter 2 loading [wt.%]']

display(df)

,STY [mgMeOH h-1 gcat-1],Family,Metal Loading [wt.%],CR Metal [pm],Support 1,MW Support 1 [g mol-1],Name of Support2,MW of Support 2 [g mol-1],Name of Support 3,MW Support 3 [g mol-1],...,Promoter 2 loading [wt.%],Calcination Temperature [K],Calcination duration [h],SBET [m2 g-1],H2/CO2 [-],GHSV [cm3 h-1 gcat-1],Catalyst amount [g],Pressure [Mpa],Temperature [K],Support Loading [wt.%]
0,2620.0,In2O3,4.4,144,Fe3O4,159.69,0,0,0,0.0,...,0.2,623,2.0,9.0,10.0,115500.0,0.50,6.0,598.0,88.6
1,2400.0,In2O3,4.4,144,Fe3O4,159.69,0,0,0,0.0,...,0.2,623,2.0,9.0,8.0,94500.0,0.50,6.0,598.0,88.6
2,2233.6,Cu,36.5,138,ZnO,81.38,0,0,SBA-15,60.0,...,0.0,573,4.0,218.0,3.0,120000.0,0.02,4.0,453.0,60.8
3,2230.0,In2O3,4.4,144,Fe3O4,159.69,0,0,0,0.0,...,0.2,623,2.0,9.0,10.0,115500.0,0.50,4.0,598.0,88.6
4,2224.0,Cu,36.5,138,ZnO,81.38,SBA-15,60,0,0.0,...,0.0,773,5.0,218.0,3.0,60000.0,0.04,4.0,453.0,60.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,382.4,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,24000.0,0.10,4.0,473.0,33.0
1230,887.2,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,42000.0,0.10,4.0,493.0,33.0
1231,1119.0,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,42000.0,0.10,4.0,513.0,33.0
1232,440.0,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,3.0,24000.0,0.10,3.0,513.0,33.0


In [27]:
X = df.iloc[:, 0]

display(X)

0       2620.0
1       2400.0
2       2233.6
3       2230.0
4       2224.0
         ...  
1229     382.4
1230     887.2
1231    1119.0
1232     440.0
1233      31.0
Name: STY [mgMeOH h-1 gcat-1], Length: 1234, dtype: float64

In [28]:
y = df[["Metal Loading [wt.%]", "Support Loading [wt.%]", "Promoter 1 loading [wt.%]", "Promoter 2 loading [wt.%]", "SBET [m2 g-1]", "H2/CO2 [-]", "GHSV [cm3 h-1 gcat-1]", "Catalyst amount [g]", "Pressure [Mpa]", "Temperature [K]", "Calcination Temperature [K]", "Calcination duration [h]"]]

# rename the columns
column_names = ["M", "S", "PR1", "PR2", "S$_{BET}$", "H$_2$/CO$_2$", "GHSV", "MC", "P", "T", "CT", "CD"]
y.columns = column_names

display(y)

,M,S,PR1,PR2,S$_{BET}$,H$_2$/CO$_2$,GHSV,MC,P,T,CT,CD
0,4.4,88.6,6.8,0.2,9.0,10.0,115500.0,0.50,6.0,598.0,623,2.0
1,4.4,88.6,6.8,0.2,9.0,8.0,94500.0,0.50,6.0,598.0,623,2.0
2,36.5,60.8,2.7,0.0,218.0,3.0,120000.0,0.02,4.0,453.0,573,4.0
3,4.4,88.6,6.8,0.2,9.0,10.0,115500.0,0.50,4.0,598.0,623,2.0
4,36.5,60.8,2.7,0.0,218.0,3.0,60000.0,0.04,4.0,453.0,773,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1229,67.0,33.0,0.0,0.0,96.0,4.0,24000.0,0.10,4.0,473.0,623,3.0
1230,67.0,33.0,0.0,0.0,96.0,4.0,42000.0,0.10,4.0,493.0,623,3.0
1231,67.0,33.0,0.0,0.0,96.0,4.0,42000.0,0.10,4.0,513.0,623,3.0
1232,67.0,33.0,0.0,0.0,96.0,3.0,24000.0,0.10,3.0,513.0,623,3.0


In [ ]:
#split into 85:15
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.15, random_state=10)


